# Recriando a classe de extração de features

A classe de extração criada anteriormente entrava em uma pasta cheia de WAVs, janelava cada um e depois extraia as features de cada janela. 

Agora, para rodar na Raspberry, eu preciso que a classe receba uma única janela, pois a leitura em tempo real vai gravar um sinal do tamanho de uma única janela, entãoa classe não precisa fazer janelamento. Além disso, eu não quero perder tempo gravando arquivos WAV e abrindo de novo, então a classe tem que receber o sinal como um array, e não o caminho para um arquivo WAV.

Para realizar os testes, já janelei algumas amostras do SESA com o Jupyter "Janelando amostras do SESA.ipynb".

Além disso, como já foi analisado, só precisamos extrair 58 features, as outras não tem importância e não vãoa crescentar nada em acurácia.

Essa classe que eu vou criar agr de "extração de features de uma única janela" não vai poder normalizar as features, essa etapa tem que ser externa, pois o "fit" do StandarScaler vai ser feito em outro lugar.

In [1]:
import librosa
import soundfile
import numpy as np
from IPython.display import Audio

/home/pi/.pyenv/versions/3.6.1/envs/venv1/lib/python3.6/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


## Definição de funções

In [2]:
def extrairRMS(sinal, frameLength, overlapLength):
    return librosa.feature.rms(y=sinal, frame_length=frameLength, hop_length=overlapLength)

In [3]:
def extrairCentroideEspectral(sinal, freqAmostragem, frameLength, overlapLength):
    return librosa.feature.spectral_centroid(y=sinal, sr=freqAmostragem, n_fft=frameLength, hop_length=overlapLength)

In [4]:
def extrairLarguraBanda(sinal, freqAmostragem, frameLength, overlapLength):
    return librosa.feature.spectral_bandwidth(y=sinal, sr= freqAmostragem, n_fft=frameLength, hop_length=overlapLength)

In [5]:
def extrairPlanicidade(sinal, frameLength, overlapLength):
    return librosa.feature.spectral_flatness(y=sinal, n_fft=frameLength, hop_length=overlapLength)

In [6]:
def extrairRolloff(sinal, freqAmostragem, frameLength, overlapLength):
    return librosa.feature.spectral_rolloff(y=sinal, sr= freqAmostragem, n_fft=frameLength, hop_length=overlapLength)

In [7]:
def extrairZCR(sinal, frameLength, overlapLength):
    return librosa.feature.zero_crossing_rate(y=sinal, frame_length=frameLength, hop_length=overlapLength)

In [8]:
def extrairMatrizMFCC(sinal, freqAmostragem):
    return librosa.feature.mfcc(y=sinal, sr=freqAmostragem)

In [9]:
def extrairMFCCs(matrizMFCC):
    
    arrayMFCCs = []

    for linha in matrizMFCC:
        arrayMFCCs.append(np.mean(linha))

    return arrayMFCCs

In [10]:
def extrairMelEspectrograma(sinal, freqAmostragem):    
    matrizMelEspectrograma = librosa.feature.melspectrogram(y=sinal, sr=freqAmostragem, n_mels=20)

    arrayMelEspectrograma = []

    for linha in matrizMelEspectrograma:
        arrayMelEspectrograma.append(np.mean(linha))

    return arrayMelEspectrograma

In [11]:
def extrairCromagramas(sinal, freqAmostragem, frameLength, overlapLength):

    matrizCromagramas = librosa.feature.chroma_stft(y=sinal, sr=freqAmostragem, n_fft=frameLength, hop_length=overlapLength)

    arrayCromagramas = []

    for linha in matrizCromagramas:
        arrayCromagramas.append(np.mean(linha))

    return arrayCromagramas

In [12]:
def extrairFeaturesUnicoFrame(sinal, freqAmostragem):
    
    # PARA IMPEDIR QUE O LIBROSA CONTINUE FAZENDO O JANELAMENTO DO ÁUDIO MESMO QUE frameLength 
    # SEJA DO TAMANHO DO ÁUDIO, O PARÂMETRO DE OVERLAP DEVE SER MAIOR QUE frameLength
    frameLength   = len(sinal)
    overlapLength = frameLength + 1

    # CRIANDO O ARRAY DE FEATURES DO FRAME EM QUESTAO
    arrayFeaturesFrame = []

    #PRIMEIRO, VOU EXTRAIR AS FEATURES UNITARIAS
    arrayFeaturesFrame.append(float(extrairRMS(sinal, frameLength, overlapLength)))
    arrayFeaturesFrame.append(float(extrairCentroideEspectral(sinal, freqAmostragem, frameLength, overlapLength)))
    arrayFeaturesFrame.append(float(extrairLarguraBanda(sinal, freqAmostragem, frameLength, overlapLength)))
    arrayFeaturesFrame.append(float(extrairPlanicidade(sinal, frameLength, overlapLength)))
    arrayFeaturesFrame.append(float(extrairRolloff(sinal, freqAmostragem, frameLength, overlapLength)))
    arrayFeaturesFrame.append(float(extrairZCR(sinal, frameLength, overlapLength)))
    arrayFeaturesFrame.extend(extrairMFCCs(extrairMatrizMFCC(sinal, freqAmostragem)))
    arrayFeaturesFrame.extend(extrairMelEspectrograma(sinal, freqAmostragem))
    arrayFeaturesFrame.extend(extrairCromagramas(sinal, freqAmostragem, frameLength, overlapLength))
    
    return arrayFeaturesFrame

## Testando as funções

Mesmo que elas já tenham sido testadas pois são uma cópia das funções criadas na classe antiga, vou testar tudo de novo.

#### Abrindo o áudio

In [13]:
caminhoJanelaTeste = "/home/pi/GravacoesReSpeaker/exemplos_amostras_SESA_v2_16kHz_16bits/exemplos_janelados_200ms/casual_000_janela_0.wav"

In [14]:
arquivoSF = soundfile.SoundFile(caminhoJanelaTeste)
print("Frequência de amostragem:", arquivoSF.samplerate)
print("Canais:", arquivoSF.channels)
print("Profundidade de bits:", arquivoSF.subtype, "\n")

Frequência de amostragem: 16000
Canais: 1
Profundidade de bits: PCM_16 



In [15]:
janelaTeste, freqAmostragem = librosa.load(caminhoJanelaTeste, sr=None, mono=True)
Audio(data=janelaTeste, rate=freqAmostragem)

#### Ajustando os parâmetros iniciais

Para impedir que o librosa faça janelamento, o overlap deve ser maior que o tamanho da janela.

In [16]:
frameLength   = len(janelaTeste)
overlapLength = frameLength + 1
print(frameLength)
print(overlapLength)

3200
3201


#### RMS

In [17]:
float(extrairRMS(janelaTeste, frameLength, overlapLength))

0.032420892268419266

#### Centroide Espectral

In [18]:
float(extrairCentroideEspectral(janelaTeste, freqAmostragem, frameLength, overlapLength))

1945.8804002039033

#### Largura de Banda

In [19]:
float(extrairLarguraBanda(janelaTeste, freqAmostragem, frameLength, overlapLength))

2374.7151391992575

#### Planicidade

In [20]:
float(extrairPlanicidade(janelaTeste, frameLength, overlapLength))

0.014923984184861183

#### Rolloff

In [21]:
float(extrairRolloff(janelaTeste, freqAmostragem, frameLength, overlapLength))

5305.0

#### Zero Crossing Rate

In [22]:
float(extrairZCR(janelaTeste, frameLength, overlapLength))

0.055

#### MFCCs

In [23]:
extrairMFCCs(extrairMatrizMFCC(janelaTeste, freqAmostragem))

[-200.0313,
 97.77553,
 -10.689477,
 -2.0161183,
 -17.110914,
 -0.21986239,
 1.9039408,
 6.5028396,
 -1.1135428,
 7.6465216,
 8.733477,
 13.215707,
 9.471103,
 9.4107,
 0.34962478,
 -1.9859645,
 1.4241152,
 6.4334707,
 6.0011935,
 3.543927]

#### Mel Espectrograma

In [24]:
extrairMelEspectrograma(janelaTeste, freqAmostragem)

[0.49913216,
 0.98430496,
 4.346118,
 17.102589,
 22.101652,
 6.0109134,
 6.4385834,
 1.4624974,
 1.0270661,
 0.19284275,
 0.06397422,
 0.096201405,
 0.11442464,
 0.07502705,
 0.046299674,
 0.03431705,
 0.008178866,
 0.0043067117,
 0.0020665547,
 0.00066244433]

#### Cromagramas

In [25]:
extrairCromagramas(janelaTeste, freqAmostragem, frameLength, overlapLength)

/home/pi/.pyenv/versions/3.6.1/envs/venv1/lib/python3.6/site-packages/librosa/core/pitch.py:146: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


[0.6457609,
 1.0,
 0.5774398,
 0.34695727,
 0.32613677,
 0.35276887,
 0.32734415,
 0.29868507,
 0.23632409,
 0.17671679,
 0.25815994,
 0.3496138]

#### Extração de tudo

In [26]:
features = extrairFeaturesUnicoFrame(janelaTeste, freqAmostragem)
print("Qtd de features:", len(features), "\n")
print(features)

Qtd de features: 58 

[0.032420892268419266, 1945.8804002039033, 2374.7151391992575, 0.014923984184861183, 5305.0, 0.055, -200.0313, 97.77553, -10.689477, -2.0161183, -17.110914, -0.21986239, 1.9039408, 6.5028396, -1.1135428, 7.6465216, 8.733477, 13.215707, 9.471103, 9.4107, 0.34962478, -1.9859645, 1.4241152, 6.4334707, 6.0011935, 3.543927, 0.49913216, 0.98430496, 4.346118, 17.102589, 22.101652, 6.0109134, 6.4385834, 1.4624974, 1.0270661, 0.19284275, 0.06397422, 0.096201405, 0.11442464, 0.07502705, 0.046299674, 0.03431705, 0.008178866, 0.0043067117, 0.0020665547, 0.00066244433, 0.6457609, 1.0, 0.5774398, 0.34695727, 0.32613677, 0.35276887, 0.32734415, 0.29868507, 0.23632409, 0.17671679, 0.25815994, 0.3496138]


# Testando o arquivo extracaoFeatures

Todas as funções implementadas acima foram colocadas no arquivo extracaoFeatures.py. ESSE ARQUIVO NÃO É UMA CLASSE. A ideia é usar funções soltas mesmo. O ARQUIVO NÃO IMPORT O NUMPY NEM O LIBROSA, ENTÃO TEM QUE IMPORTAR ESSAS BIBLIOTECAS ANTES.

#### Limpando absolutamente tudo da memória

In [27]:
import sys
sys.modules[__name__].__dict__.clear()

#### Começando do zero

In [39]:
import sys
import librosa
import numpy as np

sys.path.append("/home/pi/Programming/IC2019/Raspberry/Python/")
from extracaoFeatures import extrairFeaturesUnicoFrame as extrairFeatures

In [29]:
caminhoJanelaTeste = "/home/pi/GravacoesReSpeaker/exemplos_amostras_SESA_v2_16kHz_16bits/exemplos_janelados_200ms/casual_000_janela_0.wav"
janelaTeste, freqAmostragem = librosa.load(caminhoJanelaTeste, sr=None, mono=True)
features = extrairFeatures(janelaTeste, freqAmostragem)
print(len(features))
print(features)

58
[ 3.24208923e-02  1.94588040e+03  2.37471514e+03  1.49239842e-02
  5.30500000e+03  5.50000000e-02 -2.00031296e+02  9.77755280e+01
 -1.06894770e+01 -2.01611829e+00 -1.71109142e+01 -2.19862387e-01
  1.90394080e+00  6.50283957e+00 -1.11354280e+00  7.64652157e+00
  8.73347664e+00  1.32157068e+01  9.47110271e+00  9.41069984e+00
  3.49624783e-01 -1.98596454e+00  1.42411518e+00  6.43347073e+00
  6.00119352e+00  3.54392695e+00  4.99132156e-01  9.84304965e-01
  4.34611797e+00  1.71025887e+01  2.21016521e+01  6.01091337e+00
  6.43858337e+00  1.46249735e+00  1.02706611e+00  1.92842752e-01
  6.39742166e-02  9.62014049e-02  1.14424638e-01  7.50270486e-02
  4.62996736e-02  3.43170501e-02  8.17886554e-03  4.30671172e-03
  2.06655473e-03  6.62444334e-04  6.45760894e-01  1.00000000e+00
  5.77439785e-01  3.46957266e-01  3.26136768e-01  3.52768868e-01
  3.27344149e-01  2.98685074e-01  2.36324087e-01  1.76716790e-01
  2.58159935e-01  3.49613786e-01]


## Medindo o tempo para extrair features

Vou usar o arquivo criado para extrair features de todas as janelas exemplo de 200 ms.

In [30]:
import os
import time

In [36]:
# DIRETORIO ONDE ESTAO OS ARQUIVOS DE 200ms
dirOrigem = "/home/pi/GravacoesReSpeaker/exemplos_amostras_SESA_v2_16kHz_16bits/exemplos_janelados_200ms/"

# MEDINDO O TEMPO
arrayTempos = []
for arquivoAtual in os.listdir(dirOrigem):
    
    # ABRINDO O ARQUIVO (ISSO NAO PODE CONTAR NO TEMPO)
    sinalAtual, freqAmostragem = librosa.load(dirOrigem + arquivoAtual, sr=None, mono=True)
    
    # COMECANDO A MEDIR O TEMPO
    tempoInicio = time.time()
    
    # EXTRAINDO AS FEATURES
    features = extrairFeatures(sinalAtual, freqAmostragem)
    
    # PARANDO DE MEDIR O TEMPO
    tempoFim = time.time()
    
    # GUARDANDO O TEMPO NO ARRAY DE TEMPOS
    arrayTempos.append(tempoFim - tempoInicio)

/home/pi/.pyenv/versions/3.6.1/envs/venv1/lib/python3.6/site-packages/librosa/core/pitch.py:146: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


In [41]:
print("Média de tempo para extraír as 58 features de um sinal de 200ms:", np.mean(arrayTempos), "+-", np.std(arrayTempos))

Média de tempo para extraír as 58 features de um sinal de 200ms: 0.11583673606798486 +- 0.0017386148276367348


**Resultado:**

Média de tempo para extraír as 58 features de um sinal de 200ms com a Raspberry: 0.115837 +- 0.00174 segundos.

**Recaptulando o resultado de tempo de classificação:**

Tempo médio para classificar uma única amostra (janela de 200ms) com a Raspberry: 5.7849987321845055e-06

Isso significa que para extrair as features de uma janela de 200 ms e classificá-la, a Raspberry leva em média 0.1158425 segundos, ou 115.843 ms. Ou seja, ainda sobra 84.169 ms para fazer o Delay and Sum.